In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

kkkurumiii_summary_path = kagglehub.dataset_download('kkkurumiii/summary')

print('Data source import complete.')


In [ ]:
# part1部分
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re
from sklearn.metrics import f1_score
# 设置GPU训练
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 使用0.5B模型做ner
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
# 设置分词器
tokenizer = AutoTokenizer.from_pretrained(model_name)
# 生成式，由于我们任务是设计prompt指导大模型输出，本质是生成式的任务
# 所以用AutoModelForCausalLM（后面generate方法）
# torch_dtype往低设置为torch.float16，降低gpu显存占用，to(device)移动到gpu上训练
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)

# 定义核心的predict函数
def predict_ner(text):
    # 设计自己的prompt（其实设计了非常多的版本，但是中间的试错部分就不放了，这一版是实验下来效果最好的）
    # 观察到test.txt里面13300多个字只有不到30个book类，类别极其不平均
    # 这里直接让大模型把book类忽略了，只考虑per loc ofi，少考虑一个类别，让大模型把所有注意力
    # 都放在这三个大类上，效果更佳。
    # 采用CoT技术，首先……其次……最后……，指引模型一步一步做事。
    # 采用few-shot技术，给模型一些例子并规范输出格式。规范输出格式非常重要，否则完全无法解析输出。
    prompt = (
        "请识别以下文本中的人的名字（PER）、地方的名字（LOC）、机构的名字（OFI）。"
        "输出格式为：\n"
        "PER：[人名1, 人名2]\nLOC：[地名1, 地名2]\nOFI：[机构名1, 机构名2]\n\n"
        "首先请找出文本中的人名（PER）并列出。\n"  # Chain of Thought
        "其次找出地方名称（LOC）并列出。\n"
        "最后找出机构名称（OFI）并列出。\n\n"
        "示例：\n"  # few-shot示例
        "文本：张三去了北京的清华大学。\n"
        "PER：[张三]\nLOC：[北京]\nOFI：[清华大学]\n\n"
        "文本：王五和李四在上海合作。\n"
        "PER：[王五, 李四]\nLOC：[上海]\nOFI：[]\n\n"
        f"你需要标注的文本如下：\n{text}\n"
    )
    # 通过分词器传入prompt作为输入
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    # 输出，采用generate方法，限制最大输出tokens数为200，不要太大（速度极慢）也不能太小，怕一句话生成不完。
    # 束搜索，num_beams设为5，增强模型能力。
    # 观察模型输出，会有很多重复，他会反复输出某同一句话，因此用repetition_penalty
    outputs = model.generate(**inputs, max_new_tokens=200, num_beams=5,
        repetition_penalty=1.3, temperature=0.2, early_stopping=True)
    # 解析输出模型的预测
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return prediction

# 解析输出函数
def parse_ner_output(output):
    # set类用于去重，解析模型输出，把模型输出的三个类别的词语解析出来
    entities = {"PER": set(), "LOC": set(), "OFI": set()}
    # 采用正则表达式方式解析，因此规范模型输出格式极其重要！否则根本解析不出来任何东西
    pattern = r"PER：\s*\[(.*?)\]\s*LOC：\s*\[(.*?)\]\s*OFI：\s*\[(.*?)\]"
    # findall方式再去重
    matches = re.findall(pattern, output, re.S)
    # 模型智力不行，有时候会把input的例子也输出，因此设置default来去掉那些无关部分
    # 当然有时候去除不干净，但是不影响，查找如”清华大学“并不在文本中，就直接忽略了。
    default_phrases = {'人名1, 人名2', '地名1, 地名2', '机构名1, 机构名2',
                       '）、地方的名字（', '）、人的名字（', '）、机构的名字（',
                       '人名1', '人名2', '地名1', '地名2', '机构名1', '机构名2',
                       '张三', '李四', '王五', '清华大学', '北京大学', '上海', '北京'}
    # 按分隔符解析出每个词语放入entities中返回
    for match in matches:
        for entity_type, match_group in zip(entities.keys(), match):
            entities[entity_type].update(
                e.strip() for e in re.split(r"[，,、]", match_group)
                if e.strip() and e not in default_phrases
            )

    return {k: list(v) for k, v in entities.items()}

# 将entities转换为labels，BIEOS类型
def convert_entities_to_labels(entities, words):
    # 初始化所有标签为“O”，不属于任何实体
    labels = ["O"] * len(words)
    words = list(words)
    # 遍历entities（找到的所有实体）的所有元素
    for entity_type, entity_list in entities.items():
        for entity in entity_list:
            # 将实体按单字分割 （格式规范）
            entity_tokens = list(entity.replace(" ", ""))
            # 如果该实体长度只有1 （应该标注为S类）
            if len(entity_tokens) == 1:
                # 查找该实体在原文何处，找到后将这个位置的label设为S-某类型
                for i, word in enumerate(words):
                    if word == entity_tokens[0]:
                        labels[i] = f"S-{entity_type}"
            # 如果实体长度大于1，BIE类型设置
            elif len(entity_tokens) > 1:
                for i in range(len(words) - len(entity_tokens) + 1):
                    # 字符串匹配（暴力方法）查找该实体在原文何处。
                    if words[i:i + len(entity_tokens)] == entity_tokens:
                        labels[i] = f"B-{entity_type}"  # 实体的开始设为B
                        for j in range(1, len(entity_tokens) - 1):
                            labels[i + j] = f"I-{entity_type}"  # 实体的中间设为I
                        labels[i + len(entity_tokens) - 1] = f"E-{entity_type}"  # 实体的结束E
                        break
    return labels

# 评估函数，调用f1_score函数进行评估，对比预测和真实labels差异，平均方法选用macro（实验要求）
def evaluate_ner(predictions, labels):
    all_preds = []
    all_labels = []
    for pred, true in zip(predictions, labels):
        all_preds.extend(pred)
        all_labels.extend(true)
    f1_macro = f1_score(all_labels, all_preds, average="macro")
    return f1_macro

# 读取文件函数，由于test.txt文件中是相当于有分句的，每两个句子之间他空了一行
# 且每个句子长度适中，gpu显存可以支持，不会长度非常不平均。
# 且没行是单个字和对应的label，中间空格隔开，根据以上规则分词即可，将所有word label放入sentence
# 最后把所有sentence放入sentences（总共的数据集）并返回
def read_ner_file(filename):
    with open(filename, "r", encoding="utf-8") as f:
        sentences = []
        sentence = []
        for line in f:
            if line.strip() == "":
                if sentence:
                    sentences.append(sentence)
                    sentence = []
            else:
                word, label = line.strip().split()
                sentence.append((word, label))
        if sentence:
            sentences.append(sentence)
    return sentences

sentences = read_ner_file("./ner.txt")
# 初始化预测和标签
predictions = []
labels = []

# no_grad方式，不更新，直接用大模型对每个sentence做预测
with torch.no_grad():
    for sentence in sentences:
        words, true_labels = zip(*sentence)
        # 规范为一句连续的话
        text = " ".join(words)
        # 传入模型进行预测，得到预测输出
        pred_output = predict_ner(text)
        print(pred_output)
        # 解析为label
        pred_entities = parse_ner_output(pred_output)
        print(pred_entities)
        pred_labels = convert_entities_to_labels(pred_entities, words)
        print(pred_labels)
        # 汇总进入预测和真实标签，等待最终计算整个f1-macro值
        predictions.append(pred_labels)
        labels.append(true_labels)

# 计算f1-macro值输出，作为评测指标。
f1_macro = evaluate_ner(predictions, labels)
print(f"F1-macro: {f1_macro:.4f}")

# part1部分总结：
# 最终f1-macro部分只有0.1204，效果其实不佳。
# 可以预见。本身这个大模型感觉智力有点缺陷，我的prompt不管怎么调整他有时候输出就是完全不按照我的格式要求来。
# 当然差不多一半轮数的输出他基本按照格式了，但是还有很多预测错误。
# 剩余部分他基本完全不按照格式输出，比如有时候不输出右边中括号，有时候一个实体反复输出，
# 有时候per输出十遍……总之有很多完全无法解析的情况，造成最终效果极差。
# 只要格式稍微好一些，即使预测率不高，f1-macro都能显著上升。
# 具体可见补充实验，见文件22307110187-谢志康-3B.ipynb，采用qwen3B模型做完全相同的实验
# 输出格式正常许多，f1-macro值接近0.3了。
# 除此以外还尝试了："meta-llama/Llama-2-7b-chat-hf"模型，最终f1-macro也达到了0.2818（见简单对比.docx）
# 只需要源代码的model_name改为"meta-llama/Llama-2-7b-chat-hf"即可，实验结果可复现。
# 与上次实验对比：上次使用biLSTM网络结构，在train.txt和dev.txt上训练验证，
# 采用这个相同的test.txt做测试，微调参数之后macro能接近0.9（见简单对比.docx），
# 代码见22307110187-谢志康-pj1对比.py文件。训练轮数200。代码中均有简单分析（基本就是上次lab的内容）
# 这个对比还是非常大，其实也比较正常了，首先这个0.5B模型智力就不太行导致这边效果非常差。
# 然后biLSTM模型还是很强的，我加了很多模块，而且训练非常充分，并且是专门做ner任务了。
# 有点大模型和小专家模型对比的感觉。
# 当然用大模型做下游任务还是有好处的，不用训练，只需要设计prompt让大模型输出就行了，非常方便。

# ps：也采取过finetune大模型的方法，重训练，用wandb，但是最终没啥效果，就不说了。

cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 

The secret `HF_TOKEN` does not exist in your Colab secrets.

To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.

You will be able to reuse this secret in all of your notebooks.

Please note that authentication is recommended but still optional to access public models or datasets.

  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Streaming output truncated to the last 5000 lines.

LOC：[]

OFI：[]至 德 初 ， 取 巂 州 及 威 武 等 诸 城 ， 入 屯 石 堡 。 其 明 年 ， 使 使 来 请 讨 贼 且 脩 好 。 肃 宗 遣 给 事 中 南 巨 川 报 聘 。 然 岁 内 侵 ， 取 廓 、 霸 、 岷 等 州 及 河 源 、 莫 门 军。PER：[]

LOC：[]

OFI：[]至 德 初 ， 取 巂 州 及 威 武 等 诸 城 ， 入 屯 石 堡 。 �

{'PER': ['人名2', '李四'], 'LOC': ['地名2'], 'OFI': ['机构名2']}

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

请识别以下文本中的人的名字（PER）、地方的名字（LOC）、机构的名字（OFI）。输出格式为：

PER：[人名1, 人名2]

LOC：[地名1, 地名2]

OFI：[机构名1, 机构名2]



首先请找出文本中的人名（PER）并列出。

其次找出地方名称（LOC）并列出。

最后找出机构名称（OFI）并列出。



示例：

文本：张三去了北京的清华大学。

PER：[张三]

LOC：[北京]

OFI：[清华大学]



文本：王五和李四在上海合作。

PER：[王五, 李四]

LOC：[上海]

OFI：[]



你需要标注的文本如下：

时 责 众 官 献 便 宜 ， 议 者 以 为 宜 修 庠 序 ， 卹 典 刑 ， 审 官 方 ， 明 黜 陟 ， 举 逸 拔 才 ， 务 

In [ ]:
# part2第一次实验
# 抽取式生成文本 0.5B
# 最终成绩：Average ROUGE-1 F1 Score: 0.26356318560598013
#         Average ROUGE-2 F1 Score: 0.18321737082805475
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
# 加载rouge库，计算rouge1，rouge2的值
from rouge import Rouge
import re

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 加载模型和分词器
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)  # 不指定设备
tokenizer = AutoTokenizer.from_pretrained(model_name)

# kaggle给了两个gpu可以用，用多个GPU，使用DataParallel
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = torch.nn.DataParallel(model)  # 将模型包装为多GPU并行模型

# 将模型移动到GPU
model = model.to(device)

# 读取数据集
dataset_path = "/kaggle/input/summary/summary.jsonl"
dataset = []

with open(dataset_path, 'r', encoding='utf-8') as file:
    for line in file:
        dataset.append(json.loads(line))

# 提取输出的摘要部分
def extract_summary(text):
    # 正则表达式提取摘要，因为prompt最后是”请给出摘要：“，他会一起输出，所以这之后的内容即是模型输出的摘要
    match = re.search(r"请给出摘要：\s*(.*)", text)
    if match:
        return match.group(1)  # 返回匹配的摘要部分
    else:
        return None  # 如果没有找到摘要，返回None

# 整理格式。
def clean_summary(summary):
    # 去掉中括号，只保留其中的内容，因为reference_summary有些地名莫名其妙打了中括号
    # 这格式的差异会降低rouge值，因此规范一下
    summary = re.sub(r'\[|\]', '', summary)
    # 多个连续空格换成一个空格
    summary = re.sub(r'\s+', ' ', summary)
    return summary

# 为计算最终平均的rouge1 rouge2值
score1 = []
score2 = []

# 遍历数据集并生成摘要
for item in dataset:
    # 将文章内容合并成一个长字符串
    article = " ".join(item["article"])
    # 处理某两句过长的情况，避免oom错误（有两句实在太长了，跑到那里就oom）
    if len(article) > 2300:
        continue
    # 设计prompt，也要规范其格式，使用CoT方式优化prompt
    # 观察到jsonl数据集里的摘要都很精简，不超过50字，所以首先规范一下字数
    # 并且要求模型要给出非常精炼的summary
    # 其次观察到大部分summary是以地名开头的，如第一句：[ 连城县 ] 服务员 失手 关灯 ,……
    # 基本上大致格式是某地某人做某事，因此规范一下模型的摘要格式。
    # 观察到模型输出经常把很多数字或统计数据放进去，还有时间等，模型都会输出，因此要求不要输出这些内容
    # 最后，采用抽取式文本摘要方式的生成输出，并且要求每个词语之间必须用空格隔开
    # 就是这一步模型有时候就不理解，他部分句子输出是有隔开的，但很多都不隔开，直接输出一整句话
    # 导致rouge计算为0。这点在最终计算rouge值时作为误差舍去了。
    prompt = f"请阅读以下给定的文本后生成摘要。摘要的要求如下：\n" \
             f"1. 摘要内容必须简洁且一定不超过50个字。\n" \
             f"2. 摘要的形式为“某个地方，某人做了事情1，某人做了事情2……”，" \
             f"其中”地方“是具体的名词，”某人“是人物。\n" \
             f"3. 摘要中不要包含时间等无意义的数据。\n" \
             f"4. 摘要的内容只能使用给定文本中出现的词语，且词语之间必须用空格隔开。\n" \
             f"以下是你需要做摘要的文本：\n\n{article}\n\n" \
             f"请给出摘要：\n"
    # 取出参考的summary
    reference_summary = item["summary"]
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    # 使用模型生成输出，最大生成60tokens，比较快速，且比较精炼，束搜索设为5，与part1部分相同
    outputs = model.module.generate(**inputs, max_new_tokens=60, num_beams=5,
                             repetition_penalty=1.7, temperature=0.2, early_stopping=True)
    # 解码生成的ID序列为文本
    model_summary = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    model_summary = extract_summary(model_summary)
    # 规范参考summary和模型预测summary的格式
    reference_summary = clean_summary(reference_summary)
    model_summary = clean_summary(model_summary)
    # 打印每轮结果便于观察
    print(f"Article ID: {item['id']}")
    print(f"Reference Summary: {reference_summary}")
    print(f"Generated Summary: {model_summary}")
    # 计算ROUGE分数，加载ROUGE评估指标
    rouge = Rouge()
    rouge_score = rouge.get_scores(model_summary, reference_summary)
    s1 = rouge_score[0]["rouge-1"]["f"]
    s2 = rouge_score[0]["rouge-2"]["f"]
    print("ROUGE-1 F1 Score:", s1)
    print("ROUGE-2 F1 Score:", s2)
    print("=" * 80)
    # 排除少数几个测试模型自身不稳定导致的误差（词语之间不空格的情况）
    if s1 > 0:
        score1.append(s1)
    if s2 > 0:
        score2.append(s2)

# 最终计算ROUGE的平均值
print("Average ROUGE-1 F1 Score:", sum(score1) / len(score1))
print("Average ROUGE-2 F1 Score:", sum(score2) / len(score2))


Using device: cuda
Using 2 GPUs
Article ID: 0
Reference Summary:  连城县 服务员 失手 关灯 , 男子 冲动 之下 打砸 出气
Generated Summary: 连城县 法院 依法 审结 了 此案
ROUGE-1 F1 Score: 0.12499999531250018
ROUGE-2 F1 Score: 0.0
Article ID: 1
Reference Summary:  环翠区 七旬 老人 与 妻子 补拍 婚纱照 , 妻子 身 患 尿毒症 老人 不离不弃 ( 图 )
Generated Summary: 宫 宗良 和 姜翠 卿 在 拍 婚纱照 。 为 让 患病 老伴 开心 , 他 和 她 一起 参加 补拍 婚纱照 活动
ROUGE-1 F1 Score: 0.1666666618055557
ROUGE-2 F1 Score: 0.05263157407202262
Article ID: 2
Reference Summary: 扬州 一对 夫妻 停车 后 忙 抢 手机 红包 , 将 3岁 女儿 和 老人 锁 车内 , 祖孙 俩 深夜 敲 窗 求救
Generated Summary: 杨 女士 说 , 丈夫 也是 位 85后 , 平时 就 喜欢 玩 抢红包 的 游戏 , 一路上 开着车 红包 的 提醒 就 响 个 不停 ,
ROUGE-1 F1 Score: 0.08888888389135831
ROUGE-2 F1 Score: 0.0
Article ID: 3
Reference Summary:  卧龙区 中年男子 和 家人 发生 矛盾 , 持 菜刀 割腕 自杀 , 看到 流血 害怕 大声 呼救
Generated Summary: 和家人 生气 独自 喝 起 了 闷酒 。 之后 , 越 想 越 生气 的 李某 拿起 菜刀 企图 割腕 自杀 。 当 看到
ROUGE-1 F1 Score: 0.27027026536157783
ROUGE-2 F1 Score: 0.051282046443129975
Article ID: 4
Reference Summary: 华硕 全球 首款 骁龙 821 手机 曝光 : 5 . 7 吋 屏 , 金属 一体机 身 , 隐藏式 

In [ ]:
# part2第二次实验
# 生成式文本摘要 0.5B
# 最终成绩：Average ROUGE-1 F1 Score: 0.3559464162033218
#         Average ROUGE-2 F1 Score: 0.20571805087323114

import json
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
# 加载rouge库，计算rouge1，rouge2的值
from rouge import Rouge
import re

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 加载模型和分词器
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)  # 不指定设备
tokenizer = AutoTokenizer.from_pretrained(model_name)

# kaggle给了两个gpu可以用，用多个GPU，使用DataParallel
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = torch.nn.DataParallel(model)  # 将模型包装为多GPU并行模型

# 将模型移动到GPU
model = model.to(device)

# 读取数据集
dataset_path = "/kaggle/input/summary/summary.jsonl"
dataset = []

with open(dataset_path, 'r', encoding='utf-8') as file:
    for line in file:
        dataset.append(json.loads(line))

# 提取输出的摘要部分
def extract_summary(text):
    # 正则表达式提取摘要，因为prompt最后是”请给出摘要：“，他会一起输出，所以这之后的内容即是模型输出的摘要
    match = re.search(r"请给出摘要：\s*(.*)", text)
    if match:
        return match.group(1)  # 返回匹配的摘要部分
    else:
        return None  # 如果没有找到摘要，返回None

# 整理格式。
def clean_summary(summary):
    # 去掉中括号，只保留其中的内容，因为reference_summary有些地名莫名其妙打了中括号
    # 这格式的差异会降低rouge值，因此规范一下
    summary = re.sub(r'\[|\]', '', summary)
    # 多个连续空格换成一个空格
    summary = re.sub(r'\s+', ' ', summary)
    # 去除所有空格等，改为每个tokens之间隔一个空格
    summary = re.sub(r"([^\s])", r"\1 ", summary)
    # 去掉多余的尾部空格
    summary = summary.strip()
    return summary

# 为计算最终平均的rouge1 rouge2值
score1 = []
score2 = []

# 遍历数据集并生成摘要
for item in dataset:
    # 将文章内容合并成一个长字符串
    article = " ".join(item["article"])
    # 处理某两句过长的情况，避免oom错误（有两句实在太长了，跑到那里就oom）
    if len(article) > 2500:
        continue
    # 基本与上相同的prompt设计方式，主要是采用生成时方式的生成输出，
    # 最终clean规范格式改为每个tokens之间都空一格。
    prompt = f"请阅读以下给定的文本后生成文本摘要。摘要的要求如下：\n" \
             f"1. 摘要内容必须简洁凝练不超过50个字。\n" \
             f"2. 摘要的基本形式为“某个地方，某人做了事情1，某人做了事情2……”，" \
             f"其中”地方“是具体的地方名词，”某人“是人物。\n" \
             f"3. 摘要中不要包含任何时间、日期等无意义的数据。\n" \
             f"以下是你需要做摘要的文本：\n\n{article}\n\n" \
             f"请给出摘要：\n"
    # 取出参考的summary
    reference_summary = item["summary"]
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    # 使用模型生成输出，最大生成60tokens，比较快速，且比较精炼，束搜索设为5，与part1部分相同
    outputs = model.module.generate(**inputs, max_new_tokens=60, num_beams=5,
                             repetition_penalty=1.7, temperature=0.2, early_stopping=True)
    # 解码生成的ID序列为文本
    model_summary = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    model_summary = extract_summary(model_summary)
    # 规范参考summary和模型预测summary的格式
    reference_summary = clean_summary(reference_summary)
    model_summary = clean_summary(model_summary)
    # 打印每轮结果便于观察
    print(f"Article ID: {item['id']}")
    print(f"Reference Summary: {reference_summary}")
    print(f"Generated Summary: {model_summary}")
    # 计算ROUGE分数，加载ROUGE评估指标
    rouge = Rouge()
    rouge_score = rouge.get_scores(model_summary, reference_summary)
    s1 = rouge_score[0]["rouge-1"]["f"]
    s2 = rouge_score[0]["rouge-2"]["f"]
    print("ROUGE-1 F1 Score:", s1)
    print("ROUGE-2 F1 Score:", s2)
    print("=" * 80)
    # 排除少数几个测试模型自身不稳定导致的误差（词语之间不空格的情况）
    if s1 > 0:
        score1.append(s1)
    if s2 > 0:
        score2.append(s2)

# 最终计算ROUGE的平均值
print("Average ROUGE-1 F1 Score:", sum(score1) / len(score1))
print("Average ROUGE-2 F1 Score:", sum(score2) / len(score2))

Using device: cuda
Using 2 GPUs
Article ID: 0
Reference Summary: 连 城 县  服 务 员  失 手  关 灯  ,  男 子  冲 动  之 下  打 砸  出 气
Generated Summary: 2 0 1 3 年 9 月 , 吴 某 和 罗 某 等 人 到 连 城 县 莲 峰 镇 某 休 闲 吧 包 厢 喝 酒 时 , 因 服 务 员 在 大 厅 的 灯 时 把 所 有 灯 关 掉 , 使 得 其 中 一 个 朋 友 起 身 时 碰 到 了 啤 酒 瓶 并 砸 到 脚 。 吴 某 得 知
ROUGE-1 F1 Score: 0.24096385164174774
ROUGE-2 F1 Score: 0.08602150200023137
Article ID: 1
Reference Summary: 环 翠 区  七 旬  老 人  与  妻 子  补 拍  婚 纱 照  ,  妻 子  身  患  尿 毒 症  老 人  不 离 不 弃  (  图  )
Generated Summary: 宫  宗 良  和  姜 翠  卿  在  拍  婚 纱 照  。  为  让  患 病  老 伴  开 心  ,  他  和  她  一 起  参 加  补 拍  婚 纱 照  活 动
ROUGE-1 F1 Score: 0.31034482260998814
ROUGE-2 F1 Score: 0.13114753599570028
Article ID: 2
Reference Summary: 扬 州  一 对  夫 妻  停 车  后  忙  抢  手 机  红 包  ,  将  3 岁  女 儿  和  老 人  锁  车 内  ,  祖 孙  俩  深 夜  敲  窗  求 救
Generated Summary: 杨  女 士  说  ,  丈 夫  也 是  位  8 5 后  ,  平 时  就  喜 欢  玩  抢 红 包  的  游 戏  ,  一 路 上  开 着 车  红 包  的  提 醒  就  响  个  不 停  ,
ROUGE-1 F1 Score: 0.27777777278163585
ROUGE-2 F1 Score: 0.0259740209951097

In [ ]:
# part2第三次实验
# 生成式文本摘要 3B
# 最终成绩：Average ROUGE-1 F1 Score: 0.4202524031549726
#         Average ROUGE-2 F1 Score: 0.25937324410844187

import json
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
# 加载rouge库，计算rouge1，rouge2的值
from rouge import Rouge
import re

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 加载模型和分词器
model_name = "Qwen/Qwen2.5-3B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)  # 不指定设备
tokenizer = AutoTokenizer.from_pretrained(model_name)

# kaggle给了两个gpu可以用，用多个GPU，使用DataParallel
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = torch.nn.DataParallel(model)  # 将模型包装为多GPU并行模型

# 将模型移动到GPU
model = model.to(device)

# 读取数据集
dataset_path = "/kaggle/input/summary/summary.jsonl"
dataset = []

with open(dataset_path, 'r', encoding='utf-8') as file:
    for line in file:
        dataset.append(json.loads(line))

# 提取输出的摘要部分
def extract_summary(text):
    # 正则表达式提取摘要，因为prompt最后是”请给出摘要：“，他会一起输出，所以这之后的内容即是模型输出的摘要
    match = re.search(r"请给出摘要：\s*(.*)", text)
    if match:
        return match.group(1)  # 返回匹配的摘要部分
    else:
        return None  # 如果没有找到摘要，返回None

# 整理格式。
def clean_summary(summary):
    # 去掉中括号，只保留其中的内容，因为reference_summary有些地名莫名其妙打了中括号
    # 这格式的差异会降低rouge值，因此规范一下
    summary = re.sub(r'\[|\]', '', summary)
    # 多个连续空格换成一个空格
    summary = re.sub(r'\s+', ' ', summary)
    # 去除所有空格等，改为每个tokens之间隔一个空格
    summary = re.sub(r"([^\s])", r"\1 ", summary)
    # 去掉多余的尾部空格
    summary = summary.strip()
    return summary

# 为计算最终平均的rouge1 rouge2值
score1 = []
score2 = []

# 遍历数据集并生成摘要
for item in dataset:
    # 将文章内容合并成一个长字符串
    article = " ".join(item["article"])
    # 处理某两句过长的情况，避免oom错误（有两句实在太长了，跑到那里就oom）
    if len(article) > 2500:
        continue
    # 基本与上相同的prompt设计方式，主要是采用生成时方式的生成输出，
    # 最终clean规范格式改为每个tokens之间都空一格。
    prompt = f"请阅读以下给定的文本后生成文本摘要。摘要的要求如下：\n" \
             f"1. 摘要内容必须简洁凝练，不超过50个字。\n" \
             f"2. 摘要的基本形式为“某个地方，某人做了事情1，某人做了事情2……”，" \
             f"其中”地方“是具体的地方名词，”某人“是人物。\n" \
             f"3. 摘要中不要包含任何时间、日期等无意义的数据。\n" \
             f"以下是你需要做摘要的文本：\n\n{article}\n\n" \
             f"请给出摘要：\n"
    # 取出参考的summary
    reference_summary = item["summary"]
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    # 使用模型生成输出，最大生成60tokens，比较快速，且比较精炼，束搜索设为5，与part1部分相同
    outputs = model.generate(**inputs, max_new_tokens=55, num_beams=2,
                             repetition_penalty=1.7, temperature=0.2, early_stopping=True)
    # 解码生成的ID序列为文本
    model_summary = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    model_summary = extract_summary(model_summary)
    # 规范参考summary和模型预测summary的格式
    reference_summary = clean_summary(reference_summary)
    model_summary = clean_summary(model_summary)
    # 打印每轮结果便于观察
    print(f"Article ID: {item['id']}")
    print(f"Reference Summary: {reference_summary}")
    print(f"Generated Summary: {model_summary}")
    # 计算ROUGE分数，加载ROUGE评估指标
    rouge = Rouge()
    rouge_score = rouge.get_scores(model_summary, reference_summary)
    s1 = rouge_score[0]["rouge-1"]["f"]
    s2 = rouge_score[0]["rouge-2"]["f"]
    print("ROUGE-1 F1 Score:", s1)
    print("ROUGE-2 F1 Score:", s2)
    print("=" * 80)
    # 排除少数几个测试模型自身不稳定导致的误差（词语之间不空格的情况）
    if s1 > 0:
        score1.append(s1)
    if s2 > 0:
        score2.append(s2)

# 最终计算ROUGE的平均值
print("Average ROUGE-1 F1 Score:", sum(score1) / len(score1))
print("Average ROUGE-2 F1 Score:", sum(score2) / len(score2))

# part2部分总结：
# 整体上还是使用prompt优化模型输出，进行了以上三轮实验。prompt的设计依旧采用CoT方式
# 最终0.5B模型的成绩rouge2达到了0.2，rouge1离0.4还差一点
# Average ROUGE-1 F1 Score: 0.3559464162033218
# Average ROUGE-2 F1 Score: 0.20571805087323114
# rouge计算方式是使用了rouge库
# 考虑到输出时他会把给定的input也一同输出出来，因此最后一句话“请给出摘要：”后即为模型的输出
# 和part1基本一样的思路，解析输出后用rouge函数计算分数即可。
# 最终计算一个总的平均值。
# 使用qwen3B模型时，平均rouge1过0.4 rouge2过0.2：
# Average ROUGE-1 F1 Score: 0.4202524031549726
# Average ROUGE-2 F1 Score: 0.25937324410844187

Using device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Article ID: 0
Reference Summary: 连 城 县  服 务 员  失 手  关 灯  ,  男 子  冲 动  之 下  打 砸  出 气
Generated Summary: 连 城 县 吴 某 等 人 因 服 务 员 关 灯 问 题 与 服 务 员 发 生 争 执 后 将 休 闲 吧 内 财 物 损 毁
ROUGE-1 F1 Score: 0.31372548535178785
ROUGE-2 F1 Score: 0.1999999952000001
Article ID: 1
Reference Summary: 环 翠 区  七 旬  老 人  与  妻 子  补 拍  婚 纱 照  ,  妻 子  身  患  尿 毒 症  老 人  不 离 不 弃  (  图  )
Generated Summary: 宫  宗 良  和  姜 翠  卿  在  拍  婚 纱 照  。  为  让  患 病  老 伴  开 心  ,  他  和  她  一 起  参 加  补 拍  婚 纱 照
ROUGE-1 F1 Score: 0.32142856643494905
ROUGE-2 F1 Score: 0.1355932153404196
Article ID: 2
Reference Summary: 扬 州  一 对  夫 妻  停 车  后  忙  抢  手 机  红 包  ,  将  3 岁  女 儿  和  老 人  锁  车 内  ,  祖 孙  俩  深 夜  敲  窗  求 救
Generated Summary: 江 苏 扬 州 , 顾 女 士 深 夜 听 到 隔 壁 车 位 有 人 敲 窗 , 发 现 老 人 和 孩 子 被 锁 在 车 内 。 原 来 车 主 忙 着 抢 红 包 , 将 老 人 和 孩 子 丢 在 了 车 里 。 邻 居 及 时 发 现 并 喊 他 们 , 否 则 后 果 不 堪 设 想 。
ROUGE-1 F1 Score: 0.42553191021955633
ROUGE-2 F1 Score: 0.15094339174083315
Article ID: 3
Reference Summary: 卧 龙 区  中 年 男 子  和  家 人  发 生  矛 盾  ,  持  菜 刀  割 腕  